In [ ]:
# DELETE 문은 데이터베이스 테이블에서 행(Row)을 삭제할 때 사용
# 특정 조건에 맞는 행만 삭제할 수 있으며, 조건이 없으면 테이블의 모든 데이터가 삭제

# 사용법

# DELETE FROM table_name
# WHERE 조건;

In [ ]:
# 조건 작성시 사용하는 연산자

# 연산자,               설명,                               예시
# =,                   같다,                                age = 30
# != 또는 <>,           같지 않다,                           status != 'Pending'
# >,                   크다 (초과),                         salary > 50000
# <,                   작다 (미만),                         price < 100
# >=,                  크거나 같다 (이상),                   quantity >= 10
# <=,                  작거나 같다 (이하),                   date <= '2025-01-01'

# 연산자,               설명,                               예시
# AND,                 두 조건 모두 참일 때,                 city = 'Seoul' AND age < 30
# OR,                  둘 중 하나라도 참일 때,               country = 'KR' OR country = 'JP'
# NOT,                 조건을 부정할 때,                     NOT (price > 100)

# 연산자,               설명,                               예시
# BETWEEN,             값의 범위에 포함될 때,                amount BETWEEN 10 AND 50
# IN,                  값 목록 중 하나와 일치할 때,           department IN ('IT', 'HR')
# LIKE,                문자열 패턴이 일치할 때                name LIKE 'A%' (A로 시작)
#                      (%는 임의의 문자열, _는 임의의 한 문자)",
# IS NULL,             값이 NULL일 때,                      address IS NULL
# IS NOT NULL,         값이 NULL이 아닐 때,                  email IS NOT NULL

In [ ]:
import duckdb
import pandas as pd

con = duckdb.connect(database=':memory:')

# 테이블 생성
con.execute("""
CREATE TEMP TABLE employees_temp (
    employee_id INTEGER,
    name VARCHAR,
    department VARCHAR,
    salary INTEGER,
    is_active BOOLEAN,
    hire_year INTEGER
);
""")

# 데이터 삽입 (6개 행)
con.execute("""
INSERT INTO employees_temp VALUES
(101, 'Alice', 'HR', 60000, TRUE, 2020),
(102, 'Bob', 'IT', 85000, TRUE, 2018),
(103, 'Charlie', 'Sales', 55000, TRUE, 2022),
(104, 'David', 'IT', 70000, FALSE, 2023),
(105, 'Eve', 'HR', 72000, TRUE, 2019),
(106, 'Frank', 'Sales', 58000, FALSE, 2024);
""")


print(con.sql("SELECT * FROM employees_temp;").df())


   employee_id     name department  salary  is_active  hire_year
0          101    Alice         HR   60000       True       2020
1          102      Bob         IT   85000       True       2018
2          103  Charlie      Sales   55000       True       2022
3          104    David         IT   70000      False       2023
4          105      Eve         HR   72000       True       2019
5          106    Frank      Sales   58000      False       2024


In [3]:
print("HR 부서 직원 삭제")
con.execute("DELETE FROM employees_temp WHERE department = 'HR';")

# 삭제 후 상태 확인 (4개 행 남음)
print(con.sql("SELECT * FROM employees_temp;").df())

HR 부서 직원 삭제
   employee_id     name department  salary  is_active  hire_year
0          102      Bob         IT   85000       True       2018
1          103  Charlie      Sales   55000       True       2022
2          104    David         IT   70000      False       2023
3          106    Frank      Sales   58000      False       2024


In [4]:
print("고연봉 직원 삭제")
con.execute("DELETE FROM employees_temp WHERE salary > 70000;")

# 삭제 후 상태 확인 (3개 행 남음: Charlie, David, Frank)
print(con.sql("SELECT * FROM employees_temp;").df())


고연봉 직원 삭제
   employee_id     name department  salary  is_active  hire_year
0          103  Charlie      Sales   55000       True       2022
1          104    David         IT   70000      False       2023
2          106    Frank      Sales   58000      False       2024


In [5]:
print("Sales이거나 비활성 직원 삭제")
con.execute("DELETE FROM employees_temp WHERE department = 'Sales' OR is_active = FALSE;")

# 삭제 후 상태 확인 (0개 행 남음)
print(con.sql("SELECT * FROM employees_temp;").df())

con.close()

Sales이거나 비활성 직원 삭제
Empty DataFrame
Columns: [employee_id, name, department, salary, is_active, hire_year]
Index: []


In [ ]:
##################################################

In [7]:

con = duckdb.connect(database=':memory:')

# project_tasks 테이블 생성
con.execute("""
CREATE TEMP TABLE project_tasks (
    task_id INTEGER,
    task_name VARCHAR,
    priority INTEGER,
    due_date DATE,
    is_completed BOOLEAN
);
""")

# 데이터 삽입 (6개 행)
con.execute("""
INSERT INTO project_tasks VALUES
(1, 'Database Schema Design', 1, '2025-10-30', FALSE),
(2, 'Backend API Implementation', 2, '2025-11-15', FALSE),
(3, 'Frontend UI Mockup', 3, '2025-10-25', TRUE),
(4, 'Test Case Writing', 4, '2025-11-01', FALSE),
(5, 'Deploy to Staging', 1, '2025-11-20', FALSE),
(6, 'Documentation Review', 5, '2025-10-28', TRUE);
""")

# 확인
print("\n[초기 상태: 6개 행]")
print(con.sql("SELECT * FROM project_tasks;").df())



[초기 상태: 6개 행]
   task_id                   task_name  priority   due_date  is_completed
0        1      Database Schema Design         1 2025-10-30         False
1        2  Backend API Implementation         2 2025-11-15         False
2        3          Frontend UI Mockup         3 2025-10-25          True
3        4           Test Case Writing         4 2025-11-01         False
4        5           Deploy to Staging         1 2025-11-20         False
5        6        Documentation Review         5 2025-10-28          True


In [8]:
print("낮은 우선순위 작업 삭제 (WHERE priority BETWEEN 3 AND 5)")
con.execute("DELETE FROM project_tasks WHERE priority BETWEEN 3 AND 5;")

# 삭제 후 상태 확인 (3개 행 남음: priority 1, 2인 작업만 남음)
print(con.sql("SELECT * FROM project_tasks;").df())


낮은 우선순위 작업 삭제 (WHERE priority BETWEEN 3 AND 5)
   task_id                   task_name  priority   due_date  is_completed
0        1      Database Schema Design         1 2025-10-30         False
1        2  Backend API Implementation         2 2025-11-15         False
2        5           Deploy to Staging         1 2025-11-20         False


In [9]:

# 날짜 및 논리 연산자 (AND, <)
print("특정 날짜 이전 미완료 작업 삭제 (WHERE due_date < '2025-11-05' AND is_completed = FALSE)")

# 현재 남아있는 작업 중:
# - Task 1 (2025-10-30, FALSE) -> 삭제 대상
# - Task 2 (2025-11-15, FALSE) -> 제외 (날짜 조건 불만족)
# - Task 5 (2025-11-20, FALSE) -> 제외 (날짜 조건 불만족)
con.execute("DELETE FROM project_tasks WHERE due_date < '2025-11-05' AND is_completed = FALSE;")

# 삭제 후 상태 확인 (2개 행 남음: Task 2, 5)
print(con.sql("SELECT * FROM project_tasks;").df())


특정 날짜 이전 미완료 작업 삭제 (WHERE due_date < '2025-11-05' AND is_completed = FALSE)
   task_id                   task_name  priority   due_date  is_completed
0        2  Backend API Implementation         2 2025-11-15         False
1        5           Deploy to Staging         1 2025-11-20         False


In [10]:
print("핵심 작업(priority 1)이 아닌 모든 작업 삭제 (WHERE priority NOT IN (1))")

# 현재 남아있는 작업 중:
# - Task 2 (priority 2) -> 삭제 대상
# - Task 5 (priority 1) -> 제외

con.execute("DELETE FROM project_tasks WHERE priority NOT IN (1);")

# 삭제 후 상태 확인 (1개 행 남음: Task 5)
print(con.sql("SELECT * FROM project_tasks;").df())

# 연결 닫기
con.close()

핵심 작업(priority 1)이 아닌 모든 작업 삭제 (WHERE priority NOT IN (1))
   task_id          task_name  priority   due_date  is_completed
0        5  Deploy to Staging         1 2025-11-20         False
